In [ ]:
# GOOGLE COLAB SETUP
# from google.colab import drive
# drive.mount('/content/drive')

This is the beginning of the notebook.

ADM4142-A Fundamentals of Data science <br>
The goal of this notebook is to retrieve and stage the source datasets into the format used in the dimensional model for analysis.

This notebook generates the Location_dimension of the weather/tourism/economy data frame.




Rationale: <br>

Location dimension:

    Location_key (PK)
    Date_key (FK)
    location: string
    population: int

Location_key: integer enumeration of entries <br>
Date_key: key associated to Jan. 1st of the year corresponding to each entry.
(Only the year value is useful). <br>
location: 5 values (Canada, Alberta, British Columbia, Ontario, Quebec) <br>
population: annual population associated with the location and date above. <br>



In [ ]:
# imports
import pandas as pd

In [ ]:
# import datasets

# URLs of the datasets
urls = [
    'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/data/1710000501_databaseLoadingData-annualCanada-population.csv',
    'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/data/1710000501_databaseLoadingData-annualBritishColumbia-population.csv',
    'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/data/1710000501_databaseLoadingData-annualAlberta-population.csv',
    'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/data/1710000501_databaseLoadingData-annualOntario-population.csv',
    'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/data/1710000501_databaseLoadingData-annualQuebec-population.csv'
]

# Initialize an empty list to store DataFrames
dataframes = []

# Load, clean, and filter each dataset
for url in urls:
    # Load dataset
    df = pd.read_csv(url)

    # Remove apostrophes from all cells
    df = df.replace({"\"": ""}, regex=True)

    # Filter rows based on conditions
    df_filtered = df[(df['REF_DATE'] >= 1990) & (df['REF_DATE'] <= 2023) & (df['Age group'] == 'All ages')]

    # Append the filtered DataFrame to the list
    dataframes.append(df_filtered)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.head()

Now, generate the key mapping to map the REF_DATE value to the Jan. 1st date of the corresponding year in the dataset date.csv. (date.csv is the finished Date dimension from another notebook.)


In [ ]:
date_url = 'https://raw.githubusercontent.com/noobstang/cscsi4142-project-datasets/master/dimension/date.csv'
date_df = pd.read_csv(date_url)

# Filter for January 1st entries of each year
jan_1st_entries = date_df[(date_df['month'] == 1) & (date_df['day'] == 1) & (date_df['year'].between(1990, 2023))]

# Create the mapping of year to Date_key
date_key_map = pd.Series(jan_1st_entries['Date_key'].values,index=jan_1st_entries['year']).to_dict()


Data transformation

In [ ]:
# Assume date_key_map is a dictionary mapping each year to its corresponding Date_key (FK) in 'date.csv'
# Example: date_key_map = {1990: 1, 1991: 2, ..., 2023: 34}
#date_key_map = {}  # This needs to be defined based on the 'date.csv' file

# Mapping locations to a consistent format
location_map = {
    'Canada': 'Canada',
    'British Columbia': 'British Columbia',
    'Alberta': 'Alberta',
    'Ontario': 'Ontario',
    'Quebec': 'Quebec'
}

# Adding Location_key and Date_key to the DataFrame
#combined_df['Location_key'] = combined_df.groupby('GEO').ngroup() + 1  # Integer enumeration for locations
combined_df['Location_key'] = range(1, len(combined_df) + 1)
combined_df['Date_key'] = combined_df['REF_DATE'].apply(lambda x: date_key_map.get(x))

# Selecting and renaming columns to match the final dataset requirements
final_df = combined_df[['Location_key', 'Date_key', 'GEO', 'VALUE']]
final_df.columns = ['Location_key', 'Date_key', 'location', 'population']

# Note: You may need to adjust the mapping and transformation logic based on the actual structure of 'date.csv'


Testing

In [ ]:
final_df.tail(25)

In [ ]:
# export to csv
#final_df.to_csv('location.csv', index=False)
